In [5]:
import pandas as pd
import numpy as np 
from sklearn.tree import DecisionTreeRegressor

In [2]:
# Toy dataset
X = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 1, 1, 0],
    [0, 0, 0, 1]
], dtype=float)

y = np.array([5.0, 6.0, 2.0, 1.5])

In [12]:
n_samples = X.shape[0]

weights = np.ones(n_samples) / n_samples
weights

array([0.25, 0.25, 0.25, 0.25])

In [15]:
M = 3 # nuumber ofr boosting rounds
learners = [] # store weak learners
alphas = [] # store weights of weak learners

print("Actual:", y)
print("#" * 40)

for m in range(M):
    stump = DecisionTreeRegressor(max_depth=1)
    stump.fit(X, y, sample_weight=weights)
    y_pred = stump.predict(X)

    # Calculate absolute errors
    abs_errors = np.abs(y - y_pred)
    # Normalize errors to [0, 1]
    max_error = abs_errors.max()
    if max_error == 0:
        error = 0
    else:
        error = np.sum(weights * abs_errors) / max_error

    error = min(error, 0.999)  # avoid division by zero and log(0)

    # Compute alpha (amount of say)
    alpha = 0.5 * np.log((1 - error) / error)

    # Update weights: higher for samples with higher error
    weights *= np.exp(alpha * (abs_errors / max_error))
    # Normalize weights
    weights /= np.sum(weights)

    learners.append(stump)
    alphas.append(alpha)


    print(f"Round {m+1}")
    print("  Predictions:", y_pred)
    print("  Error:", error)
    print("  Alpha:", alpha)
    print("  Updated weights:", weights)
    print("-" * 40)

Actual: [5.  6.  2.  1.5]
########################################
Round 1
  Predictions: [4.02078205 2.9292715  4.02078205 2.9292715 ]
  Error: 0.5296734524913476
  Alpha: -0.05941672694935206
  Updated weights: [0.3501085  0.15029664 0.16626848 0.33332638]
----------------------------------------
Round 2
  Predictions: [4.0340285 2.8984754 4.0340285 2.8984754]
  Error: 0.5186762456486824
  Alpha: -0.037369877359770244
  Updated weights: [0.35281688 0.14761182 0.16541228 0.33415902]
----------------------------------------
Round 3
  Predictions: [4.04243744 2.87877419 4.04243744 2.87877419]
  Error: 0.5117054081222135
  Alpha: -0.02341509454467143
  Updated weights: [0.35450846 0.14593152 0.16485816 0.33470186]
----------------------------------------


In [16]:
# Weighted sum of learner predictions
final_pred = np.zeros(n_samples)
for alpha, learner in zip(alphas, learners):
    final_pred += alpha * learner.predict(X)

# Normalize by sum of alphas (for regression averaging)
final_pred /= np.sum(alphas)

print("Final Prediction:", final_pred)


Final Prediction: [4.02911872 2.90986041 4.02911872 2.90986041]
